## Extractive Summarization

In [3]:
import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [9]:
!pip install bert-extractive-summarizer
!pip install LexRank

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


### LexRank

In [12]:
from lexrank import LexRank

2022-11-22 12:58:36.043 Lock 2800854825168 acquired on C:\ProgramData\Anaconda3\lib\site-packages\urlextract\data\tlds-alpha-by-domain.txt.lock
2022-11-22 12:58:36.075 Lock 2800854825168 released on C:\ProgramData\Anaconda3\lib\site-packages\urlextract\data\tlds-alpha-by-domain.txt.lock


In [1]:
import numpy as np
from scipy.sparse.csgraph import connected_components

def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (
        threshold is None
        or isinstance(threshold, float)
        and 0 <= threshold < 1
    ):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None',
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    while True:
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def graph_nodes_clusters(transition_matrix, increase_power=True):
    clusters = connected_nodes(transition_matrix)
    clusters.sort(key=len, reverse=True)

    centroid_scores = []

    for group in clusters:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        centroid_scores.append(eigenvector / len(group))

    return clusters, centroid_scores


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [13]:
document="""I finished up the flimsy for Kathleen Tracy's Mini Valentine Mystery today. Nice and simple. Barb's is below though she added a roses border on hers and it is cute!
I sewed with Becky a few days ago and convinced her to start Ruby Jubilee! She wasted no time in picking out fabrics and whipping these up. She was my blogging friend before I moved to Arizona and we lived close enough together out there to be in the same guild! You may remember we did the Quiltville cruise together and lots of hiking too. Now we sew together online now and then. SEW much fun! Barb finished hers up in greens. It is going to be so cool to see these three come together in three different colorways. The next issue of Quiltmaker is out digitally and so I know we have plenty of blocks to make in the next few months. I get it on Scribd which I have a free trial of. I just cannot figure out how to print from it (unless I screenshot and do it that way and it usually does not enlarge well when I try that) and though I have the patterns from Quiltmaker and could go that route, I am tempted to just wait for my actual issue to arrive. I have plenty to keep me busy!
Recently finished these two books. I liked them both.
I have been working on R Trail and so it is trimmed, stay stitched, aqua borders cut and added and final borders assembled. I am hoping to get at least two of them on tonight, more if I have the energy.
In family and health news: my daughter, son in law and granddaughter are over covid. My oldest has it again (I think this is the third time) and it looks like my mom and dad have it too. My sister thinks she had it a week or two ago. It is crazy contagious and amazing Kathy and I have not caught it, but with all my medical appointments I feel like it won't be long. Hazel starts daycare next week so we may pick it up from her (and Hannah starts a new job too) so our exposure will be greater. I have enough issues without that on top of it. Not sure if I told you guys but my RRMS was changed to PPMS. They originally thought I had PPMS, then decided I did not and now the new neuros think that was what I originally had. Of course, that is the worse type and I have been on meds that do not help the other type which explains my downward spiral somewhat. I have now started PT and OT and they are putting in for a scooter (I got a walker already) so I can get around if I need to. May your health be good!"""

In [14]:
#Split the document into sentences
sentences = nltk.sent_tokenize(document)
print("Num sentences:", len(sentences))

#Compute the sentence embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute the pair-wise cosine similarities
cos_scores = util.cos_sim(embeddings, embeddings).numpy()

#Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

#We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


#Print the 5 sentences with the highest scores
# print("\n\nSummary:")
summary_list=[]
for idx in most_central_sentence_indices[0:10]:
    summary_list.append(sentences[idx])
print(' '.join(summary_list))

Num sentences: 3
I finished up the flimsy for Kathleen Tracy's Mini Valentine Mystery today. Barb's is below though she added a roses border on hers and it is cute! Nice and simple.


In [10]:
most_central_sentence_indices

array([ 3,  0,  7, 11,  8,  6,  9, 23,  5, 10, 22, 15,  2, 18, 14, 29,  4,
       21, 13, 20,  1, 19, 26, 17, 24, 16, 28, 27, 25, 12], dtype=int64)

### BERT Custom Model

In [1]:
from transformers import *
from summarizer import Summarizer


# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

C:\ProgramData\Anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\ProgramData\Anaconda3\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_emb

In [6]:
body = """Create another table with molecule and disease columns. It should be diseases grouped within molecule "
"""
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
model(body)

C:\Users\GOWRICH1\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


'Create another table with molecule and disease columns. It should be diseases grouped within molecule "'

In [11]:
custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading conf

In [15]:
def bert_custom_model(text,custom_model=custom_model, custom_tokenizer=custom_tokenizer):        
        # Load model, model config and tokenizer via Transformers
        model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
        return model(text)

In [19]:
model('tive clinical trials for neuroendocrine tumors nct id disease cohort investigational agent phase current status phase 2 open-label study of pembrolizumab monotherapy in patients with metastatic high grade neuroendocrine tumors 02939651 stage iv g3 neuroendocrine tumor pembrolizumab 200mg every 3 weeks 2 recruiting study of pembrolizumab with lanreotide depot for gastroenteropancreatic neuroendocrine tumors (planet) 03043664 stage iv well or moderately differentiated gastroenteropancreatic neuroendocrine tumors (gep-nets) pembrolizumab 200mg iv every 3 weeks and lanreotide depot 90mg sq every 3 weeks 1/2 recruiting study of efficacy and safety of pdr001 in patients with advanced or metastatic well-differentiated non-functional neuroendocrine tumors of pancreatic gastrointestinal (gi) or thoracic origin or poorly-differentiated gastroenteropancreatic neuroendocrine carcinoma (gep-nec) 02955069 stage iv well differentiated non-functional net of gi pancreas and lungs poorly differentiated gastroenteropancreatic neuroendocrine carcinoma pdr001 dose is 400 mg infusion every 4 weeks pdr001 is a high-affinity ligand-blocking humanized igg4 antibody directed against programmed death-1 (pd-1) receptor that blocks the binding of pd-l1 and pd-l2.')

''

### SBertSummarizer

In [19]:
from summarizer.sbert import SBertSummarizer

In [20]:
model1 = SBertSummarizer('paraphrase-MiniLM-L6-v2')

2022-11-28 19:08:05.729 Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
loading configuration file C:\Users\GOWRICH1/.cache\torch\sentence_transformers\sentence-transformers_paraphrase-MiniLM-L6-v2\config.json
Model config BertConfig {
  "_name_or_path": "C:\\Users\\GOWRICH1/.cache\\torch\\sentence_transformers\\sentence-transformers_paraphrase-MiniLM-L6-v2\\",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file C:\Users\GOWRI

In [24]:
result = model1(document)
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"I finished up the flimsy for Kathleen Tracy's Mini Valentine Mystery today. I sewed with Becky a few days ago and convinced her to start Ruby Jubilee! It is going to be so cool to see these three come together in three different colorways. I just cannot figure out how to print from it (unless I screenshot and do it that way and it usually does not enlarge well when I try that) and though I have the patterns from Quiltmaker and could go that route, I am tempted to just wait for my actual issue to arrive. It is crazy contagious and amazing Kathy and I have not caught it, but with all my medical appointments I feel like it won't be long."

In [39]:
model2 = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True)

https://huggingface.co/distilbert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\GOWRICH1\.cache\huggingface\transformers\tmp78qxk5c_


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for C:\Users\GOWRICH1/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for C:\Users\GOWRICH1/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_

...

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for C:\Users\GOWRICH1/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to C:\Users\GOWRICH1\.cache\huggingface\transformers\tmptllsx4i8


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for C:\Users\GOWRICH1/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to C:\Users\GOWRICH1\.cache\huggingface\transformers\tmp2pxm1ero


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for C:\Users\GOWRICH1/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\GOWRICH1/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file ht

In [40]:
result1 = model2(body, num_sentences=3)
result1

'Brokerages expect Immunic, Inc. (NASDAQ:IMUX) to announce ($0.73) earnings per share for the current quarter, according to Zacks Investment Research . According to Zacks, analysts expect that Immunic will report full-year earnings of ($3.88) per share for the current year, with EPS estimates ranging from ($3.95) to ($3.78). Finally, HC Wainwright restated a “buy” rating and set a $25.00 price target on shares of Immunic in a research report on Monday, December 20th.'

### Term Frequency

In [41]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [49]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

In [50]:
def stem_words(words):
    stemmed_words = []
    for word in words:
        stemmed_words.append(stemmer.stem(word))
    return stemmed_words

In [44]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [47]:
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
          dict_freq[word] = words.count(word)
    return dict_freq

In [51]:
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [52]:
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

In [53]:
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

In [54]:
def tf_idf_score(tf,idf):
    return tf*idf

In [55]:
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

In [56]:
def sentence_importance(sentence,dict_freq,sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence))
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = []
    no_of_sentences = len(sentences)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
         if word.lower() not in Stopwords and word not in Stopwords and len(word)>1:
            word = word.lower() 
            word = wordlemmatizer.lemmatize(word)
            sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
    return sentence_score

In [120]:
text=""""
"Well, we returned from Alaska about a week ago and I got a parting gift...COVID! 

In all honesty, I am surprised I hadn't gotten it before since we have been doing a decent amount of travelling since 2021. I was actually beginning to think we (DW and I) were that small group to just be immune to it. Nonetheless, after feeling like crap on day two from getting home (and a fever) and then testing positive with a home test, I went to doc in the box and got the anti-viral Paxlovid. Besides a terrible metallic taste, I had no issues with it and within a couple of days, I was feeling much better. 

So, between that and being vaxxed, it has (so far) been nothing more than a bothersome cold with a little extra fatigue to go with it. DW tested negative and has felt fine...so she still might one of the ""special ones."" 

Although we both wore masks at the airports and on the planes, the sheer number of people were...well, there were a LOT. I would guess that combined with lack of mask wearing in general made catching it pretty easy to do. When we were in Alaska, we didn't partake in any of the tourist areas and only met with a couple of friends, so no large groups outside the travel dates.

I will be happy to have this behind me as we will be travelling again in about a week to Roatan. 

Side note on COVID. The doc in box didn't seem to want run another test, they took my word for it. I am not sure how this might be reported (or not reported) to the state but I think this is more evidence that the numbers are quite likely higher than being reported. Many states are only reporting numbers once a week, so there is a lag with that too. The doc also mentioned that all the cases she had seen over the last week had been travel related.

  Quote:

     Originally Posted by explanade

     

    

    Well be prepared for some sticker shock.

Car rentals are averaging over $100 a day for compact automatics.


I can't recommend the car sharing app Turo enough. The prices are easily 1/2 of what the car rental companies charge (including the full coverage insurance) and not dealing with the lines at the rental counter is absolutely fantastic."

"""

In [121]:
def termfreq_sum(text):
    tokenized_sentence = sent_tokenize(text)
    text = remove_special_characters(str(text))
    text = re.sub(r'\d+', '', text)
    tokenized_words_with_stopwords = word_tokenize(text)
    tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)
    input_user = int(input('Percentage of information to retain(in percent):'))
    no_of_sentences = int((input_user * len(tokenized_sentence))/100)
    print(no_of_sentences)
    c = 1
    sentence_with_importance = {}
    for sent in tokenized_sentence:
        sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
        sentence_with_importance[c] = sentenceimp
        c = c+1
    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
    cnt = 0
    summary = []
    sentence_no = []
    for word_prob in sentence_with_importance:
        if cnt < no_of_sentences:
            sentence_no.append(word_prob[0])
            cnt = cnt+1
        else:
            break
    sentence_no.sort()
    cnt = 1
    for sentence in tokenized_sentence:
        if cnt in sentence_no:
            summary.append(sentence)
        cnt = cnt+1
    summary = " ".join(summary)
    print("\n")
    print("Summary:")
    print(summary)

In [122]:
termfreq_sum(text)

Percentage of information to retain(in percent):30
6


Summary:
Nonetheless, after feeling like crap on day two from getting home (and a fever) and then testing positive with a home test, I went to doc in the box and got the anti-viral Paxlovid. I would guess that combined with lack of mask wearing in general made catching it pretty easy to do. Side note on COVID. The doc in box didn't seem to want run another test, they took my word for it. I am not sure how this might be reported (or not reported) to the state but I think this is more evidence that the numbers are quite likely higher than being reported. I can't recommend the car sharing app Turo enough.


### Latent Semantic Indexing (LSI)

In [4]:
!pip install streamlit --user

  Using cached streamlit-1.15.1-py2.py3-none-any.whl (10.3 MB)
  Using cached validators-0.20.0-py3-none-any.whl
  Using cached Pillow-9.3.0-1-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached rich-12.6.0-py3-none-any.whl (237 kB)
  Using cached pyarrow-10.0.0-cp37-cp37m-win_amd64.whl (19.9 MB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached watchdog-2.1.9-py3-none-win_amd64.whl (78 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)
  Using cached pydeck-0.8.0-py2.py3-none-any.whl (4.7 MB)
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached backports.zoneinfo-0.2.1-cp37-cp37m-win_amd64.whl (38 kB)


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bertopic 0.9.4 requires hdbscan>=0.8.27, which is not instal

In [5]:
import streamlit as st
from spacy import displacy

In [6]:
from gensim.summarization import summarize

C:\Users\GOWRICH1\AppData\Roaming\Python\Python37\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2022-11-22 11:23:12.287 INFO    summarizer.preprocessing.cleaner: 'pattern' package not found; tag filters are not available for English


In [7]:
def main():
    
    st.title("Text Summarizer App")
    
    activities = ["Summarize Via Text"]#, "Summazrize via URL"]
    choice = st.sidebar.selectbox("Select Activity", activities)
    
    if choice == 'Summarize Via Text':
        st.subheader("Summary using NLP")
        raw_text = st.text_area("Enter Text Here","Type here")
        summary_choice = st.selectbox("Summary Choice" , ["Gensim","Sumy Lex rank","NLTK"])
        if st.button("Summarize Via Text"):
            if summary_choice == 'Gensim':
                summary_result = summarize(raw_text)
                
            elif summary_choice == 'Sumy Lex rank':
                summary_result = summarize(raw_text)
                
            elif summary_choice == 'NLTK':
                summary_result = summarize(raw_text)
                
            
            st.write(summary_result)
            
            
#     if choice == 'Summazrize via URL':
#         st.subheader("Summarize Your URL")
#         raw_url = st.text_input("Enter URL","Type Here")
#         if st.button("Summarize"):
#             result = get_text(raw_url)
#             #st.write(result)
#             st.subheader("Summarized Text")
#             docx = sumy_summarizer(result)
            
            
            
#             html = docx.replace("\n\n" , "\n")
#             st.markdown(html,unsafe_allow_html=True)

In [8]:
if __name__ == '__main__':
    main()

2022-11-22 11:23:45.239 
  command:

    streamlit run C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [9]:
streamlit run C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py

SyntaxError: invalid syntax (<ipython-input-9-1ccbae5e87aa>, line 1)